In [4]:
import os, sys, json
import itertools, importlib
import asyncpg, asyncio, threading

from edcompanion.eddbreader import edc_dbfilereader, edc_dbfile_process
from edcompanion.edsm_api import get_edsm_info, distance_between_systems

In [5]:
pgsql_params = dict(
    dsn=os.getenv("PGSQL_URL"),
    server_settings={'search_path': "eddb"}
)
pgpool = await asyncpg.create_pool(**pgsql_params)

In [7]:
dbname_full = 'systems.json.gz'
dbname_1day = 'systems_1day.json.gz'
dbname_1week = 'systems_1week.json.gz'
dbname_2week = 'systems_2weeks.json.gz'
dbname_1month = 'systems_1month.json.gz'
dbname_6months = 'systems_6months.json.gz'

dbname_galaxy_1day = 'galaxy_1day.json.gz'

In [ ]:
dataset = []
for item in edc_dbfilereader(os.path.join('data', dbname_1day)):
    dataset.append(item)
dataset[0]

In [5]:
classifications = {s:c for s, c in zip('OBAFGKMN', range(9))}
#print(json.dumps(classifications, indent=2))

main_star_types={}
try:
    with open(os.path.join('data', 'clasifications.json'),'rt') as jsonfile:
        main_star_types.update(json.load(jsonfile))
except:
    pass

#print(json.dumps(main_star_types, indent=2))


In [7]:
if len(main_star_types) < 40:
    for item in edc_dbfilereader(os.path.join('data', dbname_2week)):
        main_star = item.get('mainStar')

        if main_star and not main_star_types.get(main_star):
            main_star_types[main_star] = len(classifications)


    try:
        with open(os.path.join('data', 'main_star_types.json'),'wt') as jsonfile:
            json.dump(main_star_types, jsonfile, indent=3)
    except:
        pass


## Create & Build

In [ ]:
#await pgpool.execute(f"DROP TABLE IF EXISTS systems;")
await pgpool.execute(f"""
    CREATE TABLE IF NOT EXISTS systems (
        id64 BIGINT NOT NULL,
        x DOUBLE PRECISION  NOT NULL,
        y DOUBLE PRECISION  NOT NULL,
        z DOUBLE PRECISION  NOT NULL,
        name TEXT NOT NULL
    );
""")


In [ ]:

async def push_records(data):
    return await pgpool.copy_records_to_table("systems", records=data)

records = []
count=0
for item in edc_dbfilereader(os.path.join('data', dbname_full), verbose=True):
    
    if not item:
        continue

    coords = item.get('coords')
    coordinates = [coords[k] for k in ['x','y','z']]
    records.append(
        [item.get("id64")] + coordinates + [item.get('name')] 
    )
    count +=1
    if count > 10000:
        qr = await pgpool.copy_records_to_table("systems", records=records)
        records = []
        count = 0

if count > 0:
    qr = await pgpool.copy_records_to_table("systems", records=records)
    records = []
    count = 0


In [ ]:
print(f"Adding indexes ...")
await pgpool.execute(f"""
    CREATE INDEX IF NOT EXISTS systems_x_idx ON systems (x);
    CREATE INDEX IF NOT EXISTS systems_y_idx ON systems (y);
    CREATE INDEX IF NOT EXISTS systems_z_idx ON systems (z); 
    CREATE INDEX IF NOT EXISTS systems_name_idx ON systems (name);
    CREATE INDEX IF NOT EXISTS systems_id64_idx ON systems (id64);
""")

In [ ]:
print("Removing duplicates by system name")
await pgpool.execute("""
    DELETE FROM systems a
    WHERE   a.ctid <> (SELECT min(b.ctid)
                     FROM   systems b
                     WHERE  a.name = b.name );"""
)

In [ ]:
print(f"Adding unique index on system name ...")
await pgpool.execute(f"""
    DROP INDEX systems_name_idx ;
    CREATE UNIQUE INDEX IF NOT EXISTS systems_name_unique ON eddb.systems (name)
""")

In [ ]:
print("Removing duplicates by system id")
await pgpool.execute("""
    DELETE FROM systems a
    WHERE   a.ctid <> (SELECT min(b.ctid)
                     FROM   systems b
                     WHERE  a.id64 = b.id64 );"""
)

In [ ]:
print(f"Adding unique index on system id ...")
await pgpool.execute(f"""
    DROP INDEX systems_id64_idx ;
    CREATE UNIQUE INDEX IF NOT EXISTS systems_id64_unique ON eddb.systems (id64)
""")

## Update

In [ ]:
def get_coordinates_from_item(item):
    coords = item.get('coords')
    return [coords[k] for k in ['x','y','z']]

async def process_data(datachunk):

    return await pgpool.executemany(
            """INSERT INTO systems (id64, x, y, z, name) 
                VALUES ($1, $2, $3, $4, $5) 
                ON CONFLICT DO NOTHING
            """, [
                [item.get("id64")] + get_coordinates_from_item(item) + [item.get('name')] 
                for item in datachunk
            ]
    )

await edc_dbfile_process(
    os.path.join('data', dbname_1month),
    process_data,
    verbose=True
)

## Galaxy



In [ ]:
dataset = []
for item in edc_dbfilereader(os.path.join('data', dbname_galaxy_1day)):
    dataset.append(item)
    if len  (dataset) > 10:
        break
dataset[0]

In [ ]:
edsm_item = get_edsm_info('Great Annihilator', verbose=True)
edsm_item

In [ ]:
print(list(edsm_item.keys()))

In [ ]:
for k in edsm_item.keys():
    if isinstance(edsm_item[k], dict):
        for s in edsm_item[k].keys():
            print(k, s)
    elif isinstance(edsm_item[k], list):
        for i in edsm_item[k]:
            if isinstance(i,dict):
                print(k,list(i.keys()))
            else:
                print(k,len(i))
    else:
        print(k, '\t', type(edsm_item[k]))

In [159]:
ignore_keys = set(['stations', 'factions', 'thargoidWar'])

def update_structure(structure,from_item):

    assert isinstance(structure, set) or (isinstance(structure, dict) and  isinstance(from_item, dict)) or (isinstance(structure, list) and  isinstance(from_item, list)), f"structure and from_item must both be dict or list but found {type(structure)} and {type(from_item)}\n  structure:{structure}\nfrom_item:{from_item}"



    if isinstance(structure, dict):

        for k, i in from_item.items():
            if k in ignore_keys:
                continue

            if k in structure:
                update_structure(structure[k], i)

            else:
                if isinstance(i, dict):
                    structure[k] = dict()
                    update_structure(structure[k], i)
                elif isinstance(i, list):
                    structure[k] = list([dict(), set()])
                    update_structure(structure[k], i)
                else:
                    structure[k] = set()
                    update_structure(structure[k], i)

    elif isinstance(structure, list):
        sample_item = structure[0]
        for item in from_item:

            if isinstance(sample_item, set) or isinstance(item, dict):
                update_structure(sample_item, item)

            else:
                sample_set = structure[1]
                update_structure(sample_set, item)

    elif isinstance((structure), set):
        structure.add(type(from_item))

    elif isinstance(from_item, str):
        #pass
        print(from_item)
        #print(structure)


In [160]:
teststructure = dict()


In [161]:
update_structure(teststructure, edsm_item)


In [ ]:
teststructure

In [ ]:
len(dataset)

In [164]:
for item in dataset:
    update_structure(teststructure, item)

In [ ]:
teststructure

In [ ]:

for item in edc_dbfilereader(os.path.join('data', dbname_galaxy_1day), verbose=True):
    update_structure(teststructure, item)


In [ ]:
teststructure

In [ ]:
list(teststructure.keys())

In [ ]:
teststructure.pop('stations')

In [174]:
def go_through_items(iterable):

    for item in iterable:
        print(item)
        

In [ ]:
go_through_items(edc_dbfilereader(os.path.join('data', dbname_1day)))

In [33]:
import time

In [ ]:
import importlib
import edcompanion.threadworker
importlib.reload(edcompanion.threadworker)

In [2]:
from edcompanion.threadworker import create_producer_from_iterable

In [8]:
producer = create_producer_from_iterable(edc_dbfilereader(os.path.join('data', dbname_1day)))

In [ ]:
producer.start()

In [ ]:
producer.get_item()

In [18]:
for item in producer.get_items():
    print(item)


In [16]:
producer.stop()

In [17]:
producer.join()